# 2.0 Notebook 2: Feature Engineering

We have now acquired a good intuition of our data. Next step involves taking a deep dive into the data. In this section, we will rectify the quality issues with the data.

### 2.1. Introduction

What do we know of Feature Engineering? The main essense of machine learning ischoosing features, what is a feature and why do we need to engineer it? Simply puting into words all machine learning algorithms use some input data to create outputs. This input data comprise features, which are usually in the form of structured columns. Algorithms require features with some specific characteristic to work properly.Therefor we require feature engineering. Main objectives :

* Prepare the accurate input dataset, compatible with the machine learning algorithm requirements 
* Improve the performance of machine learning models.

### 2.2 Objective

Since many features in our data is categorical value, we need to find appropriate techniques to convert them into integers, as our model only inputs numerical values, remove outliers,etc. Few operations to be performed here:
1. Imputation
2. Handling Outliers
3. Log Transform
4. One-Hot Encoding
5. Feature Split
6. Scaling
7. Extracting Date

> Thus, the goal here is to properly mould our dataset to fit perfectly into our model. 
